<a href="https://colab.research.google.com/github/kimsengduong/VCTK_VITS/blob/master/YourTTS_en_mv_REV5_022323_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine tune a Your-TTS model with the Coqui TTS framework using audio samples of your choice.**

Thank you to all of the [Coqui TTS](https://https://github.com/coqui-ai/TTS) contributors, the OpenAI team, whoever authored the rnnoise script, and [@Thorsten-Voice](https://https://www.youtube.com/c/ThorstenMueller ) for his helpful videos.

The Your-TTS training loop and configuration come from the training recipe by Edresson and iamkhalidbashir.  After cloning the Github repo, you can find it in TTS/recipes/vctk/yourtts/train_yourtts.py

[-nn](https://https://www.youtube.com/c/NanoNomad )

If you have run the script before, and want to examine your training session using Tensorboard, set your dataset directories, then skip to the bottom section to load Tensorboard.

This notebook is currently limited to training English voices.  If you want to try other languages, or multi-speaker datasets, you're on your own for now.



**Dependencies recently broke.  Run this cell first, then click RESTART RUNTIME within the output box below. When your session reconnects, continue on with the notebook from the Connect Google Drive cell.**

In [ ]:
!pip install TTS

**Run this cell to connect your Google Drive account. Samples, datasets and tuned models will be saved here.**

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

**Variable names**
Name your dataset/dataset directory. Single words, no spaces.

Set dataset and trainer output directory for saving model states and logs.  This will be a subdirectory of your dataset directory.  If the dataset directory does not exist, it will be created.

Set the sample upload directory, then place each voice's wav/mp3 files in separate folders, one for each voice, under this directory.

Example:
sampleuploads/<br>
->sampleuploads/bob/[all bob's audio files here]<br>
->sampleuploads/jerry/[all jerry's audio files here]<br>
->sampleuploads/tom/[all tom's audio files here]<br>

Set your run type.  Restore run begins a new fine tuning session using the model named here (downloaded using Coqui later).  Continue run continues a previous, interrupted fine tuning run (You will select the run to continue later).

In [ ]:
ds_name = "deck-ds" #@param {type:"string"}
output_directory = "traineroutput" #@param {type:"string"}
upload_dir = "yourtts-sampleuploads" #@param {type:"string"}
MODEL_FILE = "/root/.local/share/tts/tts_models--multilingual--multi-dataset--your_tts/model_file.pth" #@param {type:"string"}
upload_dir = "/content/drive/MyDrive/" + upload_dir
RUN_NAME = "YourTTS-EN-VCTK" #@param {type:"string"}

OUT_PATH = "/content/drive/MyDrive/"+ds_name+"/traineroutput/"
!mkdir $upload_dir
!mkdir /content/drive/MyDrive/$ds_name
!mkdir /content/drive/MyDrive/$ds_name/txt/
!mkdir /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/

In [ ]:
run_type = "restore" #@param ["continue","restore","restore-ckpt","newmodel"]
print(run_type + " run selected")

**Download and Build Rnnoise (https://github.com/xiph/rnnoise) and Requirements**

In [ ]:
#@title
!pip install pyloudnorm
!git clone https://github.com/xiph/rnnoise.git
!sudo apt-get install curl autoconf automake libtool python-dev pkg-config sox ffmpeg
%cd /content/rnnoise
!sh autogen.sh
!sh configure
!make clean
!make

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'rnnoise'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 420 (delta 194), reused 192 (delta 192), pack-reused 201
Receiving objects: 100% (420/420), 836.04 KiB | 4.04 MiB/s, done.
Resolving deltas: 100% (221/221), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'python-dev-is-python2' instead of 'python-dev'
autoconf is already the newest version (2.69-11.1).
autoconf set to manually installed.
automake is already the newest version (1:1.16.1-4ubuntu6).
automake set to manually installed.
pkg-config is already the newest version (0.29.1-0ubuntu4).
curl is already the newest version (7.68.0-1ubuntu2.15).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
The following package was automatically i

**Install Sox, Install OpenAI Whisper STT+Translation (https://github.com/openai/whisper)**

In [ ]:
#@title
%cd /content
!sudo apt install sox
!git clone https://github.com/openai/whisper.git
!pip install git+https://github.com/openai/whisper.git


/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
sox is already the newest version (14.4.2+git20190427-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Cloning into 'whisper'...
remote: Enumerating objects: 483, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 483 (delta 86), reused 115 (delta 70), pack-reused 343
Receiving objects: 100% (483/483), 7.50 MiB | 18.38 MiB/s, done.
Resolving deltas: 100% (285/285), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-5lkyel0p
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-5lkyel

**Install Coqui TTS** (https://github.com/coqui-ai/TTS), espeak-ng phonemeizer (https://github.com/espeak-ng/espeak-ng), download Coqui TTS source and examples from GitHub.

In [ ]:
#@title
%cd /content
!sudo apt-get install espeak-ng
!git clone https://github.com/coqui-ai/TTS.git

/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
espeak-ng is already the newest version (1.50+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
fatal: destination path 'TTS' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.21.6-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.0
    Uninstalling numpy-1.20.0:
      Successfully uninstalled numpy-1.20.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.20.0-cp38-cp38-manylinux2010_x86_64.whl (15.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.11.0 requires numpy==1.21.6; python_version < "3.10", but you have numpy 1.20.0 which is incompatible.
cmdstanpy 1.1.0 requires numpy>=1.21, but you have numpy 1.20.0 which is incompatible.


**(Optional) List pretrained models available on the Coqui Hub**

In [ ]:
!tts --list_models

 Name format: type/language/dataset/model
 1: tts_models/multilingual/multi-dataset/your_tts [already downloaded]
 2: tts_models/bg/cv/vits
 3: tts_models/cs/cv/vits
 4: tts_models/da/cv/vits
 5: tts_models/et/cv/vits
 6: tts_models/ga/cv/vits
 7: tts_models/en/ek1/tacotron2
 8: tts_models/en/ljspeech/tacotron2-DDC
 9: tts_models/en/ljspeech/tacotron2-DDC_ph
 10: tts_models/en/ljspeech/glow-tts
 11: tts_models/en/ljspeech/speedy-speech
 12: tts_models/en/ljspeech/tacotron2-DCA
 13: tts_models/en/ljspeech/vits
 14: tts_models/en/ljspeech/vits--neon
 15: tts_models/en/ljspeech/fast_pitch
 16: tts_models/en/ljspeech/overflow
 17: tts_models/en/vctk/vits
 18: tts_models/en/vctk/fast_pitch
 19: tts_models/en/sam/tacotron-DDC
 20: tts_models/en/blizzard2013/capacitron-t2-c50
 21: tts_models/en/blizzard2013/capacitron-t2-c150_v2
 22: tts_models/es/mai/tacotron2-DDC
 23: tts_models/es/css10/vits
 24: tts_models/fr/mai/tacotron2-DDC
 25: tts_models/fr/css10/vits
 26: tts_models/uk/mai/glow-tts


**List folders in sample upload directory**

Google Drive can become desyncronized when uploading files with the web interface.  If your sample folder doesn't show up here, wait, or use the desktop application.  If Colab can't see the folder, it can't access the samples.

In [ ]:
#@title
%cd $upload_dir
!ls -al

**Set the sample uploads subfolder name to process.**

Set the name for the new speaker to process.  This will be the speakerid, and stored in the model as VCTK_{name}

You can name them both the same thing

In [ ]:
subfolder = "deck" #@param {type:"string"}
newspeakername = "deck" #@param {type:"string"}

**Process**
This section will convert mp3 and wav files in upload_dir to  16000hz mono wav files.  Then it will pass the wav files through rnnoise.

rnnoise output is then segmented based on 0.2 second silences (click show code below, change 0.2 in the sox line to the duration to silence duration if needed)

8000hz Highpass and 50hz lowpass filters applied, gain/loudness adjusted to reduce potential clipping, -6db peak normalization and -25db lufs applied.  Should be fine for general purpose.

segmented audio is then passed through sox again to force-split any long segments (above 8 seconds) into segments once again.  Files smaller than 35kb are deleted.

These files are then converted to .flac and renamed to correspond to the VCTK dataset format.

**Audio Preprocessing Options**

Recommended: Leave all 'true'

run_denoise use Xiph's rnnoise on samples

run_splits split samples based on silence interval of 0.2 seconds and then force a split into 8 second segments. Click view code and find the 'sox' lines to change these intervals.

use_audio_filter engage highpass filter 50hz, lowpass fitler 8000hz. Click view code and find the 'sox' lines to change these frequencies if needed.

normalize_audio to engage -6db peak -25LUFS normalization

In [ ]:
run_denoise = "True" #@param ["True", "False"]
run_splits = "True" #@param ["True", "False"]
use_audio_filter = "True" #@param ["True", "False"]
normalize_audio = "True" #@param ["True", "False"]
#start_sil_dur = 0.2 #@param {type:"number"}
#end_sil_dur = 0.2 #@param {type:"number"}
#sample_max = 8 #@param {type:"number"}


In [ ]:
#@title

from pathlib import Path
import os
import subprocess
import soundfile as sf
import pyloudnorm as pyln
import sys
import glob
%cd $upload_dir
%cd $subfolder
#Normalize file extensions, convert all to lower case. Use -n flag to test.
#https://stackoverflow.com/a/32729737
!rename 's/\.([^.]+)$/.\L$1/' *

#!ls -al
!rm -rf $upload_dir/$subfolder/16k_1ch
!mkdir $upload_dir/$subfolder/16k_1ch

!find . -name '*.mp3' -exec bash -c 'for f; do ffmpeg -hide_banner -loglevel error -i "$f" -acodec pcm_s16le -ar 16000 -ac 1 16k_1ch/"${f%.mp3}".wav ; done' _ {} +
!find . -name '*.ogg' -exec bash -c 'for f; do ffmpeg -hide_banner -loglevel error -i "$f" -acodec pcm_s16le -ar 16000 -ac 1 16k_1ch/"${f%.ogg}".wav ; done' _ {} +
!find . -name '*.wav' -exec bash -c 'for f; do ffmpeg -hide_banner -loglevel error -i "$f" -acodec pcm_s16le -ar 16000 -ac 1 16k_1ch/"${f%.wav}".wav ; done' _ {} +
!ls -al $upload_dir/$subfolder/16k_1ch
print("Files converted to 16khz 1ch wav")
%cd $upload_dir
%cd $subfolder
!rm temp.raw
!rm rnn.raw
%cd 16k_1ch
!find . -name "*.wav" -type f -size -35k -delete
%cd $upload_dir
#Convert and resample uploaded mp3/wav clips to 1 channel, 16khz

#
if run_denoise=="True":
  print("Running denoise...")
  orig_wavs= upload_dir + '/' + subfolder + "/16k_1ch/"
  print(orig_wavs)

  from pathlib import Path
  import os
  import subprocess
  import soundfile as sf
  import pyloudnorm as pyln
  import sys
  import glob
  rnn = "/content/rnnoise/examples/rnnoise_demo"
  paths = glob.glob(os.path.join(orig_wavs, '*.wav'))
  for filepath in paths:
    base = os.path.basename(filepath)
    tp_s = upload_dir + '/' + subfolder + "/16k_1ch/denoise/"
    tf_s = upload_dir + '/' + subfolder + "/16k_1ch/denoise/" + base
    target_path = Path(tp_s)
    target_file = Path(tf_s)
    print("From: " + str(filepath))
    print("To: " + str(target_file))



  # Stereo to Mono; upsample to 48000Hz
  # added -G to fix gain, -v 0.8
    subprocess.run(["sox", "-G", "-v", "0.8", filepath, "48k.wav", "remix", "-", "rate", "48000"])
    subprocess.run(["sox", "48k.wav", "-c", "1", "-r", "48000", "-b", "16", "-e", "signed-integer", "-t", "raw", "temp.raw"]) # convert wav to raw
    subprocess.run(["/content/rnnoise/examples/rnnoise_demo", "temp.raw", "rnn.raw"]) # apply rnnoise
    subprocess.run(["sox", "-G", "-v", "0.8", "-r", "48k", "-b", "16", "-e", "signed-integer", "rnn.raw", "-t", "wav", "rnn.wav"]) # convert raw back to wav

    subprocess.run(["mkdir", "-p", str(target_path)])
    if use_audio_filter=="True":
      print("Running highpass/lowpass & resample")
      subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "highpass", "50", "lowpass", "8000", "rate", "16000"])
      # apply high/low pass filter and change sr to 16000Hz
      data, rate = sf.read(target_file)
    elif use_audio_filter=="False":
      print("Running resample without filter")
      subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "rate", "16000"])
      # apply high/low pass filter and change sr to 16kHz
      data, rate = sf.read(target_file)
# peak normalize audio to -6 dB
    if normalize_audio=="True":
      print("Output normalized")
      peak_normalized_audio = pyln.normalize.peak(data, -6.0)

# measure the loudness first
      meter = pyln.Meter(rate) # create BS.1770 meter
      loudness = meter.integrated_loudness(data)

# loudness normalize audio to -25 dB LUFS
      loudness_normalized_audio = pyln.normalize.loudness(data, loudness, -25.0)
      sf.write(target_file, data=loudness_normalized_audio, samplerate=16000)
      print("")
    elif normalize_audio=="False":
      print("File written without normalizing")
      sf.write(target_file, data=data, samplerate=16000)
      print("")

  !rm $target_path/rnn.wav
  !rm $target_path/48k.wav

elif run_denoise=="False":
  paths = glob.glob(os.path.join(orig_wavs, '*.wav'))
  for filepath in paths:
    print("Skipping denoise...")
    base = os.path.basename(filepath)
    tp_s = upload_dir + '/' + subfolder + "/16k_1ch/denoise/"
    tf_s = upload_dir + '/' + subfolder + "/16k_1ch/denoise/" + base
    target_path = Path(tp_s)
    target_file = Path(tf_s)
    print("From: " + str(filepath))
    print("To: " + str(target_file))
    subprocess.run(["sox", "-G", "-v", "0.8", filepath, "48k.wav", "remix", "-", "rate", "48000"])
    subprocess.run(["sox", "48k.wav", "-c", "1", "-r", "48000", "-b", "16", "-e", "signed-integer", "-t", "raw", "temp.raw"]) # convert wav to raw
    #subprocess.run(["/content/rnnoise/examples/rnnoise_demo", "temp.raw", "rnn.raw"]) # apply rnnoise
    subprocess.run(["sox", "-G", "-v", "0.8", "-r", "48k", "-b", "16", "-e", "signed-integer", "rnn.raw", "-t", "wav", "rnn.wav"]) # convert raw back to wav
    subprocess.run(["mkdir", "-p", str(target_path)])
    if use_audio_filter=="True":
      print("Running filter...")
      subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "highpass", "50", "lowpass", "8000", "rate", "16000"]) # apply high/low pass filter and change sr to 160Hz
      data, rate = sf.read(target_file)
    elif use_audio_filter=="False":
      print("Skipping filter...")
      subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "rate", "16000"]) # apply high/low pass filter and change sr to 22050Hz
      data, rate = sf.read(target_file)
          # peak normalize audio to -6 dB
    if normalize_audio=="True":
      print("Output normalized")
      peak_normalized_audio = pyln.normalize.peak(data, -6.0)

# measure the loudness first
      meter = pyln.Meter(rate) # create BS.1770 meter
      loudness = meter.integrated_loudness(data)

# loudness normalize audio to -25 dB LUFS
      loudness_normalized_audio = pyln.normalize.loudness(data, loudness, -25.0)
      sf.write(target_file, data=loudness_normalized_audio, samplerate=16000)
      print("")
    if normalize_audio=="False":
      print("File written without normalizing")
      sf.write(target_file, data=data, samplerate=16000)
      print("")
  !rm $target_path/rnn.wav
  !rm $target_path/48k.wav

if run_splits=="False":
  print("Copying files without splitting...")
  %mkdir /content/drive/MyDrive/$ds_name
  %mkdir /content/drive/MyDrive/$ds_name/wav48_silence_trimmed
  %mkdir /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/$newspeakername

  !cp $target_path/*.wav /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/$newspeakername
if run_splits=="True":
  %mkdir /content/drive/MyDrive/$ds_name
  %mkdir /content/drive/MyDrive/$ds_name/wav48_silence_trimmed
  %mkdir /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/$newspeakername
  print("Splitting output and copying...")
  %cd $target_path
  !rm -rf splits
  !mkdir splits
  !for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress silence 1 0.2 0.1% 1 0.2 0.1% : newfile : restart ; done
#alt split method: force splits of 9.5 seconds, however this will split words. Comment the above with # and remove the # below to change
#!for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress trim 0 8 : restart ; done
  %cd splits
  !mkdir resplit
  !for FILE in *.wav; do sox "$FILE" resplit/"$FILE" --show-progress trim 0 9 : newfile : restart ; done
  %cd resplit
  !find . -name "*.wav" -type f -size -35k -delete
  #!ls -al
  %cd /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/$newspeakername

#  !ls -al
!cp $target_path/splits/resplit/*.wav /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/$newspeakername
%cd /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/$newspeakername
!rm *.flac
!find . -name '*.wav' -exec bash -c 'for f; do ffmpeg -i "$f" -c:a flac "${f%.wav}"_mic1.flac ; done' _ {} +
!rm *.wav
!ls /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/$newspeakername

**Run Whisper on generated audio clips.**
Transcripts will be formatted for the VCTK-style dataset and placed in the<br>
->dataset directory<br>
---->txt<br>
-------->newspeakername<br>

**List dataset speaker subdirectories**

In [ ]:
#@title
!ls /content/drive/MyDrive/$ds_name/wav48_silence_trimmed

**Select Whisper STT model. Large-v2 slowest, most accurate, most memory usage. Free Colab users may need to use medium.en.  Select model, run next cell to load it.**

**Run the load cell only once. Reloading Whisper STT models may crash your Colab session.**

In [ ]:
whisper_model = "large-v2" #@param ["large-v2", "large-v1", "medium.en", "small.en", "base.en"]


Run this cell once only.

Load OpenAI Whisper large-v1 STT model to memory.  Loading it multiple times tends to crash Colab.  

Click Show Code below and swap the comment to change models if large-v1 doesn't work on your Colab session.

In [ ]:
#@title
import whisper
import os, os.path
import glob
import pandas as pd

from pathlib import Path


#model = whisper.load_model("medium.en")
model = whisper.load_model(whisper_model)

**Set the folder to process**

In [ ]:
newspeakername = "deck" #@param {type:"string"}


**Run this cell to transcribe clips using Whisper.**

To process additional speakers, set a new subfolder/newspeakername above, and remember to re-run that cell before running the one below.

In [ ]:
#@title
wavs = '/content/drive/MyDrive/'+ds_name+'/wav48_silence_trimmed/'+newspeakername


"""
if os.path.exists(modelpath):
	if os.path.isfile(modelpath+"large-v2.pt"):
		print("Saved large-v2 found")
		model = whisper.load_model(download_root=modelpath,name="large-v2")
else:
	print("loading model from Huggingface")
	model = whisper.load_model("large-v2")
"""
paths = glob.glob(os.path.join(wavs, '*.flac'))
print(len(paths))
all_filenames = []
transcript_text = []
try:
	os.mkdir('/content/drive/MyDrive/'+ds_name+'/txt/'+newspeakername+'/')
except OSError as error:
	print(error)

for filepath in paths:
	base = os.path.basename(filepath)
	all_filenames.append(base)

	result = model.transcribe(filepath)
	output = result["text"].lstrip()
	output = output.replace("\n","")
	print(output)
	thefile = str(os.path.basename(filepath).lstrip(".")).rsplit(".")[0]
	thefile = thefile[:-5]
	print(thefile)
	outfilepath = '/content/drive/MyDrive/'+ds_name+'/txt/'+newspeakername+'/'+thefile+'.txt'
	with open(outfilepath, 'w', encoding='utf-8') as indfile:
		indfile.write(output)

Perform the steps above for each new speaker.  You should have a folder arraged like this for your dataset.<br>
datasetfolder<br>
->txt<br>
---->nameone<br>
---->nametwo<br>
->wav48_silence_trimmed<br>
---->nameone<br>
---->nametwo<br>

Before continuing, download your dataset from Google Drive and listen to the audio files, and review the corresponding transcript. If you're not preprocessing with a dataloader (I'm not here), transcribe numbers by hand.

Delete any audio files that do not sound good, and their corresponding transcript

**Part 2 - Training**

Download TTS model

Load Tensorboard and dashboard

Set training variables, load trainer, train

**Download Your-TTS model**
Generates Sample Wav File to /content/vctk-your-tts.wav  This will be deleted when your Colab session is closed.**

In [ ]:
%cd /content
!tts --text "flag to one of these values to use the multi-speaker model" --language_idx en --model_name "tts_models/multilingual/multi-dataset/your_tts" --speaker_idx female-en-5 --out_path /content/vctk-your-tts.wav

/content
 > tts_models/multilingual/multi-dataset/your_tts is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > log_func:np.log10
 | > min_l

**Load Tensorboard**

In [ ]:
import torch
%load_ext tensorboard

**Load Dashboard**
May take several minutes to appear from a blank white box.  Ad blockers probably need to whitelist a bunch of Colab stuff or this won't work.

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/$ds_name/$output_directory/

**Set Variables**

In [ ]:
#@title
import os
import torch
from trainer import Trainer, TrainerArgs
from TTS.bin.compute_embeddings import compute_embeddings
from TTS.bin.resample import resample_files
from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsArgs, VitsAudioConfig
from TTS.utils.downloaders import download_vctk
from TTS.config import load_config
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.managers import save_file
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.data import get_length_balancer_weights
from TTS.tts.utils.languages import LanguageManager, get_language_balancer_weights
from TTS.tts.utils.speakers import SpeakerManager, get_speaker_balancer_weights, get_speaker_manager
#torch.set_num_threads(12)

RESTORE_PATH = MODEL_FILE  # "/root/.local/share/tts/tts_models--multilingual--multi-dataset--your_tts/model_file.pth"
SKIP_TRAIN_EPOCH = False
BATCH_SIZE = 16
SAMPLE_RATE = 16000
MAX_AUDIO_LEN_IN_SECONDS = 10
VCTK_DOWNLOAD_PATH = "/content/drive/MyDrive/"+ds_name
NUM_RESAMPLE_THREADS = 10
# Check if VCTK dataset is not already downloaded, if not download it
#if not os.path.exists(VCTK_DOWNLOAD_PATH):
#    print(">>> Downloading VCTK dataset:")
#    download_vctk(VCTK_DOWNLOAD_PATH)


**Resample Dataset.**

Back up your dataset before running.  Only run once on the dataset.

In [ ]:
#@title
resample_files(VCTK_DOWNLOAD_PATH, SAMPLE_RATE, file_ext="flac", n_jobs=NUM_RESAMPLE_THREADS)

**Initialize dataset config.**

In [ ]:
#@title
vctk_config = BaseDatasetConfig(
    formatter="vctk",
    dataset_name=ds_name,
    meta_file_train="",
    meta_file_val="",
    path=VCTK_DOWNLOAD_PATH,
    language="en-us",
    ignored_speakers=[]
)
#Add additional datasets
#DATASETS_CONFIG_LIST = [vctk_config,vctk_otherdataset,vctk_anotherdataset]
DATASETS_CONFIG_LIST = [vctk_config]

## Extract speaker embeddings
SPEAKER_ENCODER_CHECKPOINT_PATH = "https://github.com/coqui-ai/TTS/releases/download/speaker_encoder_model/model_se.pth.tar"

SPEAKER_ENCODER_CONFIG_PATH = "https://github.com/coqui-ai/TTS/releases/download/speaker_encoder_model/config_se.json"
D_VECTOR_FILES = []  # List of speaker embeddings/d-vectors to be used during the training



**(Optional, may degrade performance, may break things entirely)**

Append original vectors from Coqui YourTTS model to attempt to maintain original included speakers.

In [ ]:
D_VECTOR_FILES.append("/root/.local/share/tts/tts_models--multilingual--multi-dataset--your_tts/speakers.json")

**Iterates all the dataset configs checking if the speakers embeddings are already computated, if not compute it.  Saves file to dataset/speakers.pth**

In [ ]:
for dataset_conf in DATASETS_CONFIG_LIST:
    # Check if the embeddings weren't already computed, if not compute it
    print(dataset_conf.path)
    embbase=str(dataset_conf.dataset_name)
    #embeddings_file = MODEL_DIR+"speakers.pth"
    embeddings_file = os.path.join(dataset_conf.path, embbase+"_speakers.pth")
    print(embeddings_file)
    if not os.path.isfile(embeddings_file):
        print(f">>> Computing the speaker embeddings for the {dataset_conf.dataset_name} dataset")
        print(SPEAKER_ENCODER_CHECKPOINT_PATH)
        print(SPEAKER_ENCODER_CONFIG_PATH)
        print(embeddings_file)
        print(dataset_conf.formatter)
        print(dataset_conf.dataset_name)
        print(dataset_conf.path)
        print(dataset_conf.meta_file_train)
        print(dataset_conf.meta_file_val)
        compute_embeddings(
            SPEAKER_ENCODER_CHECKPOINT_PATH,
            SPEAKER_ENCODER_CONFIG_PATH,
            embeddings_file,
            old_spakers_file=None,
            config_dataset_path=None,
            formatter_name=dataset_conf.formatter,
            dataset_name=dataset_conf.dataset_name,
            dataset_path=dataset_conf.path,
            meta_file_train=dataset_conf.meta_file_train,
            meta_file_val=dataset_conf.meta_file_val,
            disable_cuda=False,
            no_eval=False,
        )
    D_VECTOR_FILES.append(embeddings_file)

**Set VitsAudioConfig and VitsArgs**

In [ ]:
# Audio config used in training.
audio_config = VitsAudioConfig(
    sample_rate=SAMPLE_RATE,
    hop_length=256,
    win_length=1024,
    fft_size=1024,
    mel_fmin=0.0,
    mel_fmax=None,
    num_mels=80,
)

# Init VITSArgs setting the arguments that is needed for the YourTTS model
model_args = VitsArgs(
    d_vector_file=D_VECTOR_FILES,
    use_d_vector_file=True,
    d_vector_dim=512,
    num_layers_text_encoder=10,
    speaker_encoder_model_path=SPEAKER_ENCODER_CHECKPOINT_PATH,
    speaker_encoder_config_path=SPEAKER_ENCODER_CONFIG_PATH,
    resblock_type_decoder="2",  # On the paper, we accidentally trained the YourTTS using ResNet blocks type 2, if you like you can use the ResNet blocks type 1 like the VITS model
    # Usefull parameters to enable the Speaker Consistency Loss (SCL) discribed in the paper
    use_speaker_encoder_as_loss=True,
    # Usefull parameters to the enable multilingual training
    # use_language_embedding=True,
    # embedded_language_dim=4,
)


**Tweak Config**


If training more than one speaker, enable the speaker weighted sampler.

In [ ]:
# General training config, here you can change the batch size and others usefull parameters
config = VitsConfig(
    output_path=OUT_PATH,
    model_args=model_args,
    run_name=RUN_NAME,
    project_name="YourTTS",
    run_description="""YourTTS transfer learning test""",
    dashboard_logger="tensorboard",
    logger_uri=None,
    audio=audio_config,
    batch_size=BATCH_SIZE,
    #batch_group_size=16,
    batch_group_size=0,
    eval_batch_size=BATCH_SIZE,
    num_loader_workers=2,
    eval_split_max_size=256,
    print_step=50,
    plot_step=100,
    log_model_step=500,
    save_all_best=True,
    save_step=1000,
    save_n_checkpoints=5,
    save_checkpoints=True,
    target_loss="loss_0",
    print_eval=False,
    use_phonemes=True,
    phonemizer="espeak",
    phoneme_language="en-us",
    compute_input_seq_cache=True,
    add_blank=True,
    text_cleaner="multilingual_cleaners",
    phoneme_cache_path="phoneme_cache",
    precompute_num_workers=2,
    start_by_longest=True,
    datasets=DATASETS_CONFIG_LIST,
    cudnn_benchmark=False,
    max_audio_len=SAMPLE_RATE * MAX_AUDIO_LEN_IN_SECONDS,
    mixed_precision=False,
    # Enable the weighted sampler
    # Set to false if training one speaker.
    # Set to true if training 2 or more speakers (optional)
    #use_weighted_sampler=True,
    use_weighted_sampler=False,
    # Ensures that all speakers are seen in the training batch equally no matter how many samples each speaker has
    weighted_sampler_attrs={"speaker_name": 1.0},
    weighted_sampler_multipliers={"speaker_name": {}},
    # It defines the Speaker Consistency Loss (SCL) α to 9 like the paper
    speaker_encoder_loss_alpha=9.0
)

**Manually engage the speaker manager and language manager. Export ID files to dataset directory (speaker_ids.json, speakers.json, language_ids.json. Don't know if this is necessary at the moment, but now you have them.)**

In [ ]:
#@title
speaker_manager = SpeakerManager(
    d_vectors_file_path=D_VECTOR_FILES,
    encoder_model_path=SPEAKER_ENCODER_CHECKPOINT_PATH,
    encoder_config_path=SPEAKER_ENCODER_CONFIG_PATH
)

print("Speakers: " + str(speaker_manager.get_speakers()))
speaker_manager.save_ids_to_file(VCTK_DOWNLOAD_PATH+"/speaker_ids.json")
print("Speaker IDs written to: " + VCTK_DOWNLOAD_PATH+"/speaker_ids.json")
speaker_manager.save_embeddings_to_file(VCTK_DOWNLOAD_PATH+"/speakers.json")
print("Speaker embeddings saved to: " + VCTK_DOWNLOAD_PATH+"/speakers.json")
language_manager = LanguageManager(config=config)
config.model_args.num_languages = language_manager.num_languages
print(str(language_manager.num_languages) + " languages found")
language_manager.save_ids_to_file(VCTK_DOWNLOAD_PATH+"/language_ids.json")
print("Language IDs written to: " + VCTK_DOWNLOAD_PATH+"/language_ids.json")
# Load all the datasets samples and split traning and evaluation sets
train_samples, eval_samples = load_tts_samples(
    config.datasets,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

#speaker_manager.set_ids_from_data(train_samples + eval_samples, parse_key="speaker_name")
config.num_speakers = speaker_manager.num_speakers
config.model_args.num_speakers = speaker_manager.num_speakers
config.model_args.speakers_file=None
config.speakers_file=None
config.save_json(VCTK_DOWNLOAD_PATH+"config.json")
print("Config.json written to "+VCTK_DOWNLOAD_PATH+"config.json")
#onfig.load_json(VCTK_DOWNLOAD_PATH+"config.json")
#print("Config.json loaded from "+VCTK_DOWNLOAD_PATH+"config.json")

**Generate test sentences**

In [ ]:
ALL_SPEAKERS = []
ALL_SPEAKERS = speaker_manager.speaker_names
ALL_SENTENCES = []
TEST_SENTENCES_PH1 = "I am the very model of a modern Major-General, I've information vegetable, animal, and mineral,  I know the kings of England, and I quote the fights historical. From Marathon to Waterloo, in order categorical"
TEST_SENTENCES_PH2 = "I'm sorry, but I don’t want to be an emperor. That's not my business. I don't want to rule or conquer anyone. I should like to help everyone, if possible; Jew, Gentile, black man, white. We all want to help one another. Human beings are like that. We want to live by each other's happiness - not by each other's misery. We don't want to hate and despise one another. In this world there is room for everyone. And the good earth is rich and can provide for everyone. The way of life can be free and beautiful, but we have lost the way."
TEST_SENTENCES_PH3 = "A pot of tea helps to pass the evening."
TEST_SENTENCES_PH4 = "Smoky fires lack flame and heat."
print(ALL_SPEAKERS)
for speakername in ALL_SPEAKERS:
    ph1 = [TEST_SENTENCES_PH1,speakername,None,"en-us"]
    ph2 = [TEST_SENTENCES_PH2,speakername,None,"en-us"]
    ph3 = [TEST_SENTENCES_PH1,speakername,None,"en-us"]
    ph4 = [TEST_SENTENCES_PH2,speakername,None,"en-us"]
    ALL_SENTENCES.append(ph1)
    ALL_SENTENCES.append(ph2)
    ALL_SENTENCES.append(ph3)
    ALL_SENTENCES.append(ph4)
#print(ALL_SENTENCES)
config.test_sentences=ALL_SENTENCES

['VCTK_deck']


**Initialize the model**

In [ ]:
model = Vits.init_from_config(config)

**If continuing a run, run the next cell to list all run directories. Copy and paste the run you want to continue into the next box, then run the cell.**

In [ ]:
#@title
!ls -al /content/drive/MyDrive/$ds_name/traineroutput

total 8
drwx------ 2 root root 4096 Jan 26 01:23 YourTTS-EN-VCTK-January-26-2023_01+23AM-0000000
drwx------ 2 root root 4096 Jan 26 00:57 YourTTS-EN-VCTK-January-26-2023_12+57AM-0000000


**Run to continue**

In [ ]:
run_folder = "YourTTS-EN-VCTK-January-26-2023_01+23AM-0000000" #@param {type:"string"}


**Init the trainer**

Trainer options

In [ ]:
print("Current reinit_text_encoder value: " + str(config.model_args.reinit_text_encoder))
reinit_te_status = "False" #@param ["False", "True"]
if reinit_te_status=="False":
  config.model_args.reinit_text_encoder=False
  print("Text encoder will not be reinitilized")
elif reinit_te_status=="True":
  config.model_args.reinit_text_encoder=True
  print("Model arguments set to reinitilize text encoder")
  print("Current reinit_DP value: " + str(config.model_args.reinit_DP))
reinit_DP_status = "False" #@param ["False", "True"]
if reinit_DP_status=="False":
  config.model_args.reinit_DP=False
  print("DP will not be reinitilized")
elif reinit_DP_status=="True":
  config.model_args.reinit_DP=True
  print("Model arguments set to reinitilize DP")
print("Current freeze_waveform_decoder value: " + str(config.model_args.freeze_waveform_decoder))
freeze_waveform_decoder_status = "False" #@param ["False", "True"]
if freeze_waveform_decoder_status=="False":
  print("Waveform decoder will NOT be frozen")
  config.model_args.freeze_waveform_decoder=False
elif freeze_waveform_decoder_status=="True":
  config.model_args.freeze_waveform_decoder=True
  print("Waveform decoder FROZEN")
print("Current freeze_flow_decoder value: " + str(config.model_args.freeze_flow_decoder))
freeze_flow_decoder_status = "False" #@param ["False", "True"]
if freeze_flow_decoder_status=="False":
  print("Flow decoder will NOT be frozen")
  config.model_args.freeze_flow_decoder=None
elif freeze_flow_decoder_status=="True":
  config.model_args.freeze_flow_decoder="True"
  print("Flow decoder FROZEN")
print("Current freeze_encoder value: " + str(config.model_args.freeze_encoder))
freeze_encoder_status = "False" #@param ["False", "True"]
if freeze_encoder_status=="False":
  print("Text encoder will NOT be frozen")
  config.model_args.freeze_encoder=False
elif freeze_encoder_status=="True":
  config.model_args.freeze_encoder=True
  print("Text encoder FROZEN")
print("Current freeze_DP value: " + str(config.model_args.freeze_DP))
freeze_DP_status = "False" #@param ["False", "True"]
if freeze_DP_status=="False":
  print("Duration predictor will NOT be frozen")
  config.model_args.freeze_DP=False
elif freeze_DP_status=="True":
  config.model_args.freeze_DP=True
  print("Duration predictor FROZEN")

Initilize the trainer

In [ ]:
#@title
print(run_type)

if run_type=="continue":
  CONTINUE_PATH="/content/drive/MyDrive/"+ds_name+"/traineroutput/"+run_folder
  trainer = Trainer(
    TrainerArgs(continue_path=CONTINUE_PATH, skip_train_epoch=SKIP_TRAIN_EPOCH),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
elif run_type=="restore":
    trainer = Trainer(
    TrainerArgs(restore_path=MODEL_FILE, skip_train_epoch=SKIP_TRAIN_EPOCH),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
elif run_type=="restore-ckpt":
  trainer = Trainer(
  TrainerArgs(restore_path="/content/drive/MyDrive/"+ds_name+"/traineroutput/"+run_folder+"/"+ckpt_file, skip_train_epoch=SKIP_TRAIN_EPOCH),
  config,
  output_path=OUT_PATH,
  model=model,
  train_samples=train_samples,
  eval_samples=eval_samples,
  )
elif run_type=="newmodel":
  trainer = Trainer(
  TrainerArgs(),
  config,
  output_path=OUT_PATH,
  model=model,
  train_samples=train_samples,
  eval_samples=eval_samples,
)

**Run training**

In [ ]:
trainer.fit()